In [8]:
#importing library
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import nltk
import string
from nltk.tokenize import RegexpTokenizer,sent_tokenize
from nltk.tokenize import word_tokenize


nltk.download('punkt')
import os

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
input = pd.read_excel("Input.xlsx")
input

def get_article_names(urls):
  titles = []
  for i in range (len(urls)):
    title = urls[i]
    title_clean = title[title.index( "m/" ) + 2 :-1]. replace('-' , ' ')
    titles.append(title_clean)
  return titles

urls =input["URL"]
urls_id=input["URL_ID"]
urlsTitleDF = get_article_names(urls)
urlsTitleDF

['rising it cities and its impact on the economy environment infrastructure and city life by the year 2040 2',
 'rising it cities and their impact on the economy environment infrastructure and city life in future',
 'internet demands evolution communication impact and 2035s alternative pathways',
 'rise of cybercrime and its effect in upcoming future',
 'ott platform and its impact on the entertainment industry in future',
 'the rise of the ott platform and its impact on the entertainment industry by 2040',
 'rise of cyber crime and its effects',
 'rise of internet demand and its impact on communications and alternatives by the year 2035 2',
 'rise of cybercrime and its effect by the year 2040 2',
 'rise of cybercrime and its effect by the year 2040',
 'rise of internet demand and its impact on communications and alternatives by the year 2035',
 'rise of telemedicine and its impact on livelihood by 2040 3 2',
 'rise of e health and its impact on humans by the year 2030',
 'rise of e he

In [3]:
corps =[]
for url in urls:

  page=requests.get(url , headers={"User-Agent": "XY"})  
  soup = BeautifulSoup(page.text , 'html.parser')
  #get title
  try:
    title = soup . find("h1",attrs = { 'class' : 'entry-title'}).get_text()
    text = soup . find(attrs = { 'class' : 'td-post-content'}).get_text()

  except AttributeError:
    title = "none"  
    text="none"

  #get article text

  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)
  corps.append(text)


In [5]:
len(corps)

100

# tokenizer

In [15]:
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    return tokens

In [9]:
punc=[punc for punc in string.punctuation]
StopWords_Auditor=pd.read_csv("StopWords/StopWords_Auditor.txt",header=None)
StopWords_DatesandNumbers=pd.read_csv("StopWords/StopWords_DatesandNumbers.txt",header=None)
StopWords_Generic=pd.read_csv("StopWords/StopWords_Generic.txt",header=None)
StopWords_GenericLong=pd.read_csv("StopWords/StopWords_GenericLong.txt",header=None)
StopWords_Geographic=pd.read_csv("StopWords/StopWords_Geographic.txt",header=None)
StopWords_Names=pd.read_csv("StopWords/StopWords_Names.txt",header=None)
StopWords_Currencies = pd.read_csv("StopWords/StopWords_Currencies.txt", header=None, encoding="ISO-8859-1",sep='|')
StopWords_Currencies.drop(columns=[1])
def text_process(text):
    nopunc =[char for char in text if char not in punc or char not in [':',',','(',')','’','?']]
    nopunc=''.join(nopunc)
    txt=' '.join([word for word in nopunc.split() if word.lower() not in StopWords_Auditor])
    txt1=' '.join([word for word in txt.split() if word.lower() not in StopWords_Currencies])
    txt2=' '.join([word for word in txt1.split() if word.lower() not in StopWords_DatesandNumbers])
    txt3=' '.join([word for word in txt2.split() if word.lower() not in StopWords_Generic])
    txt4=' '.join([word for word in txt3.split() if word.lower() not in StopWords_GenericLong])
    txt5=' '.join([word for word in txt4.split() if word.lower() not in StopWords_Geographic])
    return ' '.join([word for word in txt5.split() if word.lower() not in StopWords_Names])

In [11]:
positive=pd.read_csv("MasterDictionary/positive-words.txt",header=None)
with open("MasterDictionary/negative-words.txt", "r", encoding="ISO-8859-1") as file:
    lines = file.readlines()
negative = pd.DataFrame(lines, columns=["Word"])

positive.columns=['text']
negative.columns=['text']
positive['text']=positive['text'].astype(str)
negative['text']=negative['text'].astype(str)
positive['text']=positive['text'].apply(text_process)
negative['text']=negative['text'].apply(text_process)
#positive list
length=positive.shape[0]
post=[]
for i in range(0,length):
   nopunc =[char for char in positive.iloc[i] if char not in string.punctuation or char != '+']
   nopunc=''.join(nopunc)

   post.append(nopunc)
#negative list
length=negative.shape[0]
neg=[]
for i in range(0,length):
  nopunc =[char for char in negative.iloc[i] if char not in string.punctuation or char != '+']
  nopunc=''.join(nopunc)
  neg.append(nopunc)

In [22]:
#Positive Scoe
def positive_score(text):
    tokenize_text=tokenizer(text)    
    positive_score=0
    for i in tokenize_text:
        if(i.lower() in post):
            positive_score+=1
    return(positive_score)

# negative score
def negative_score(text):    
    tokenize_text=tokenizer(text)
    negative_score=0
    for i in tokenize_text:
        if(i.lower() in neg):
            negative_score+=1
    return( negative_score)

#Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
def polarity_score(pos_score,neg_score):
    Polarity_Score=(pos_score-neg_score)/((positive_score+negative_score)+0.000001)
    return(Polarity_Score)

#Subjective score
def subjective_score(positive_score,negative_score):
    subjectiivity_score=(positive_score-negative_score)/((len(tokenize_text))+ 0.000001)
    return(subjectiivity_score)

#avg sent length
def avg_sent_length(text):
        sentences = sent_tokenize(text)
        
        # Calculate the total number of words
        total_words = sum(len(sentence.split()) for sentence in sentences)
        
        # Calculate the total number of sentences
        total_sentences = len(sentences)
        
        # Calculate the average sentence length
        if total_sentences > 0:
            avg_length = total_words / total_sentences
        else:
            avg_length = 0
        
        return avg_length

#perc of complex words
def perc_of_complexwords(text):
    tokenize_text=tokenizer(text)
    vowels=['a','e','i','o','u']
    import re
    count=0
    complex_Word_Count=0
    for i in tokenize_text:
        x=re.compile('[es|ed]$')
        if x.match(i.lower()):
            count+=0
        else:
            for j in i:
                if(j.lower() in vowels ):
                    count+=1
        if(count>2):
            complex_Word_Count+=1
        count=0
    return(complex_Word_Count/len(tokenize_text))

#Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
def fog_index(avg_sentence_length,Percentage_of_Complex_words):    
    Fog_Index = 0.4 * (avg_sentence_length + Percentage_of_Complex_words)
    return(Fog_Index )

#Avg words in sentence
def avg_words_sent(text):
    words = text.split()  # Split the sentence into words
    total_words += len(words)  # Add the number of words to the total

    num_sentences = len(text)
    average_words = total_words / num_sentences
    return average_words



#complex wordcount
def Complex_wordcount(text):
    tokenize_text=tokenizer(text)    
    vowels=['a','e','i','o','u']
    import re
    count=0
    complex_Word_Count=0
    for i in tokenize_text:
        x=re.compile('[es|ed]$')
        if x.match(i.lower()):
            count+=0
        else:
            for j in i:
                if(j.lower() in vowels ):
                    count+=1
        if(count>2):
            complex_Word_Count+=1
        count=0
        return(complex_Word_Count)

#word count
def word_count(text):    
    tokenize_text=tokenizer(text)
    word_count=len(tokenize_text)
    return(word_count)    

#syllabus per word
def syllabus_per_word(text):  
    tokenize_text=tokenizer(text)  
    vowels=['a','e','i','o','u']
    import re
    count=0
    for i in tokenize_text:
        x=re.compile('[es|ed]$')
        if x.match(i.lower()):
            count+=0
        else:
            for j in i:
                if(j.lower() in vowels ):
                    count+=1
        syllable_count=count
    return(syllable_count)

#personal pronoun
def pers_pronoun(text):  
    tokenize_text=tokenizer(text)  
    pronouns=['i','we','my','ours','us' ]
    import re
    count=0
    for i in tokenize_text:
        if i.lower() in pronouns:
            count+=1
    personal_pronouns=count
    return(personal_pronouns )

#avg word length
def avg_wordlength(text):    
    tokenize_text=tokenizer(text)
    count=0
    for i in tokenize_text:
        for j in i:
            count+=1
    avg_word_length=count/len(tokenize_text)
    return(avg_word_length)

In [20]:
df=pd.DataFrame({"URL_ID":urls_id})
df["URL"]=url
df = pd.DataFrame({'corps': corps})
df["positive_score"] = df["corps"] . apply (positive_score)
df["negative_score"]=df["corps"].apply(negative_score)
df["polarity_score"] = np.vectorize(polarity_score)(df['positive_score'],df['negative_score'])
df["subjective_score"] = np.vectorize(subjective_score)(df['positive_score'],df['negative_score'])
df["avg_length_score"]=df["corps"].apply(avg_sent_length)
df["perc_of_complex_words"]=df["corps".apply(perc_of_complexwords)]
df["fog_index"]=np.vectorize(fog_index).apply(df["avg_length_score"],df["perc_of_complex_words"])
df["avg_words_in_sent"]=df["corps"].apply(avg_words_sent)
df["complex_wordcount"]=df["corps"].apply(Complex_wordcount)
df["word_count"]=df["corps"].apply(word_count)
df["syllabus_per_word"]=df["corps".apply(syllabus_per_word)]
df["personal_pronouns"]=df["corps"].apply(pers_pronoun)
df[]



In [21]:
df["negative_score"]

0      6
1     31
2     26
3     75
4      9
      ..
95    57
96    35
97     3
98     0
99     0
Name: negative_score, Length: 100, dtype: int64